In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Utils.DataReader import load_urm, load_icm, load_target

In [2]:
URM_all = load_urm()

#URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
#URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.85)
URM_train = sps.load_npz('Dataset/Split/URM_train.npz')
URM_test =  sps.load_npz('Dataset/Split/URM_test.npz')
URM_validation = sps.load_npz('Dataset/Split/URM_validation.npz')

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1569 ( 3.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 936 ( 2.2%) Users that have less than 1 test interactions


In [3]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

recommender_SLIMElasticNet = SLIMElasticNetRecommender(URM_train)
recommender_SLIMElasticNet.fit(topK=405, l1_ratio=0.0010299956370568744, alpha=0.01)

SLIMElasticNetRecommender: Processed 8991 (36.7%) in 5.00 min. Items per second: 29.96
SLIMElasticNetRecommender: Processed 18116 (73.9%) in 10.00 min. Items per second: 30.19
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 13.48 min. Items per second: 30.29


In [4]:
import pyximport
pyximport.install()
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/miniconda3/envs/tensorflow/bin/python'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from MatrixFactorizationImpressions_Cython_Epoch.c:746:
In file included from /opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:5:
In file included from /opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1948:
/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy A

Compiling [4/10]: Sparse_Matrix_Tree_CSR.pyx... PASS

Compiling [5/10]: Triangular_Matrix.pyx... 
In file included from Triangular_Matrix.c:746:
In file included from /opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:5:
In file included from /opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1948:
/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
1 warning generated.
/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, 

#warning "Using deprecated NumPy API, disable it with " \
 ^
CFW_D_Similarity_Cython_SGD.c:28563:21: warning: fallthrough annotation in unreachable code [-Wunreachable-code-fallthrough]
                    CYTHON_FALLTHROUGH;
                    ^
CFW_D_Similarity_Cython_SGD.c:341:34: note: expanded from macro 'CYTHON_FALLTHROUGH'
      #define CYTHON_FALLTHROUGH __attribute__((fallthrough))
                                 ^
CFW_D_Similarity_Cython_SGD.c:28574:21: warning: fallthrough annotation in unreachable code [-Wunreachable-code-fallthrough]
                    CYTHON_FALLTHROUGH;
                    ^
CFW_D_Similarity_Cython_SGD.c:341:34: note: expanded from macro 'CYTHON_FALLTHROUGH'
      #define CYTHON_FALLTHROUGH __attribute__((fallthrough))
                                 ^
3 warnings generated.
/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will

In [5]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

recommender_RP3beta = RP3betaRecommender(URM_train)
recommender_RP3beta.fit(alpha=0.08934428697754494, beta=0.15025275671838492, topK=132, implicit=False)
#recommender_RP3beta.fit(topK=167, alpha=1.0, beta=0.4520495673133021, implicit=True)

RP3betaRecommender: Similarity column 24507 (100.0%), 1861.94 column/sec. Elapsed time 13.16 sec


In [ ]:
recommender_SLIMElasticNet.W_sparse

In [ ]:
recommender_RP3beta.W_sparse

In [6]:
print("MAP of the starting models")

result_df, _ = evaluator_test.evaluateRecommender(recommender_SLIMElasticNet)
print("SLIM ElasticNet - MAP: {}".format(result_df.loc[10]["MAP"]))

result_df, _ = evaluator_test.evaluateRecommender(recommender_RP3beta)
print("RP3beta - MAP: {}".format(result_df.loc[10]["MAP"]))

MAP of the starting models
EvaluatorHoldout: Processed 40693 (100.0%) in 19.96 sec. Users per second: 2038
SLIM ElasticNet - MAP: 0.022279966836462446
EvaluatorHoldout: Processed 40693 (100.0%) in 13.78 sec. Users per second: 2954
RP3beta - MAP: 0.021232793635970547


In [ ]:
user_id = 0

item_scores_SLIM_EN = recommender_SLIMElasticNet._compute_item_score(user_id)
item_scores_SLIM_EN

In [ ]:
item_scores_RP3 = recommender_RP3beta._compute_item_score(user_id)
item_scores_RP3

In [ ]:
alpha = 0.5 #50/50 slim-rp3
new_similarity = (1-alpha)*recommender_RP3beta.W_sparse + alpha * recommender_SLIMElasticNet.W_sparse
new_similarity

In [7]:
from Recommenders.BaseRecommender import BaseRecommender

class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2


    def fit(self, alpha = 0.5):
        self.alpha = alpha


    def _compute_item_score(self, user_id_array, items_to_compute):

        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights

In [8]:
recommender_object = ScoresHybridRecommender(URM_train, recommender_SLIMElasticNet, recommender_RP3beta)

best_model = {
    "MAP" : 0,
    "alpha" : 0,
    "norm" : 0
}

for norm in [1,2]:
    for alpha in np.arange(0.0, 1.1, 0.1):

            #truncate digits since np.arange sometimes doesn't
            alpha = round(alpha,1)
            recommender_object.fit(norm, alpha)
            result_df, _ = evaluator_validation.evaluateRecommender(recommender_object)
            print("Norm: {}, Alpha: {}, Result: {}".format(norm, alpha, 1-alpha, result_df.loc[10]["MAP"]))

            if result_df.loc[10]["MAP"] > best_model["MAP"]:
                best_model["MAP"] = result_df.loc[10]["MAP"]
                best_model["alpha"] = alpha
                best_model["norm"] = norm

                print("*** New best model found! ")
                print("New best model has MAP: {} with alpha: {}, norm: {}".format(best_model["MAP"], best_model["alpha"], best_model["norm"]))

print("----")
print("Best model has MAP: {} with alpha: {}, norm: {}".format(best_model["MAP"], best_model["alpha"], best_model["norm"]))

EvaluatorHoldout: Processed 40060 (100.0%) in 25.33 sec. Users per second: 1582
Norm: 1, Result: 0.01919768165212472
EvaluatorHoldout: Processed 37000 (92.4%) in 5.07 min. Users per second: 122
EvaluatorHoldout: Processed 40060 (100.0%) in 5.39 min. Users per second: 124
Norm: 2, Result: 0.019198197743067625
EvaluatorHoldout: Processed 40060 (100.0%) in 23.68 sec. Users per second: 1692
Norm: inf, Result: 0.019207818232967737
EvaluatorHoldout: Processed 40060 (100.0%) in 23.69 sec. Users per second: 1691
Norm: -inf, Result: 0.01919809175126166


In [ ]:
recommender = ScoresHybridRecommender(URM_train, recommender_SLIMElasticNet, recommender_RP3beta)
recommender.fit(norm=1, alpha = 0.5)

## Submissions

In [9]:
test_users = pd.read_csv('Dataset/data_target_users_test.csv')

In [10]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender_object.recommend(user, cutoff=10))

In [11]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])

test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('Submissions\Submission_SLIM_EN_rp3Beta_URM_Binary.csv', index=False)